In [4]:
import pandas as pd

books = pd.read_csv("../data/books_with_categories.csv")

In [11]:
from transformers import pipeline

classifier = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", return_all_scores = True, device=0)
classifier("I love this")

Device set to use cpu


[[{'label': 'anger', 'score': 0.0013875309377908707},
  {'label': 'disgust', 'score': 0.0007134036859497428},
  {'label': 'fear', 'score': 0.00039849153836257756},
  {'label': 'joy', 'score': 0.9845667481422424},
  {'label': 'neutral', 'score': 0.0034753058571368456},
  {'label': 'sadness', 'score': 0.004531427752226591},
  {'label': 'surprise', 'score': 0.0049272081814706326}]]

In [6]:
books["description"][0]

'A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gilead is a song of celebration and acceptance of the best and the worst the world ha

In [7]:
classifier(books["description"][0])

[[{'label': 'anger', 'score': 0.003935154061764479},
  {'label': 'disgust', 'score': 0.019100721925497055},
  {'label': 'fear', 'score': 0.6548399925231934},
  {'label': 'joy', 'score': 0.015161462128162384},
  {'label': 'neutral', 'score': 0.1698525995016098},
  {'label': 'sadness', 'score': 0.11640939861536026},
  {'label': 'surprise', 'score': 0.02070068009197712}]]

In [12]:
sentences = books["description"][0].split(".")
predictions = classifier(sentences)

In [13]:
sentences[0]

'A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives'

In [14]:
predictions[0]

[{'label': 'anger', 'score': 0.009156374260783195},
 {'label': 'disgust', 'score': 0.002628476358950138},
 {'label': 'fear', 'score': 0.06816231459379196},
 {'label': 'joy', 'score': 0.0479426383972168},
 {'label': 'neutral', 'score': 0.14038598537445068},
 {'label': 'sadness', 'score': 0.002122163539752364},
 {'label': 'surprise', 'score': 0.729602038860321}]

In [15]:
sentences[3]

' Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist'

In [16]:
predictions[3]

[{'label': 'anger', 'score': 0.03219091519713402},
 {'label': 'disgust', 'score': 0.008431830443441868},
 {'label': 'fear', 'score': 0.9281681180000305},
 {'label': 'joy', 'score': 0.001045584212988615},
 {'label': 'neutral', 'score': 0.012808694504201412},
 {'label': 'sadness', 'score': 0.008756881579756737},
 {'label': 'surprise', 'score': 0.008597923442721367}]

In [20]:
import numpy as np

emotion_labels = ["anger", "disgust", "fear", "joy", "sadness", "surprise", "neutral"]
isbns= []
emotion_scores = {label: [] for label in emotion_labels}

def calculate_max_emotion_scores(predictions):
    per_emotion_scores = {label: [] for label in emotion_labels}
    for prediction in predictions:
        sorted_predictions = sorted(prediction, key = lambda x: x["label"])
        for index, label in enumerate(emotion_labels):
            per_emotion_scores[label].append(sorted_predictions[index]["score"])
    return {label: np.max(scores) for label, scores in per_emotion_scores.items()}

In [21]:
for i in range(10):
    isbns.append(books["isbn13"][i])
    sentences = books["description"][i].split(".")
    predictions = classifier(sentences)
    max_scores = calculate_max_emotion_scores(predictions)
    for label in emotion_labels:
        emotion_scores[label].append(max_scores[label])

In [22]:
emotion_scores

{'anger': [0.0641336515545845,
  0.6126194000244141,
  0.0641336515545845,
  0.3514845371246338,
  0.0814124271273613,
  0.23222433030605316,
  0.5381841063499451,
  0.0641336515545845,
  0.3006698489189148,
  0.0641336515545845],
 'disgust': [0.2735915184020996,
  0.34828439354896545,
  0.10400670766830444,
  0.1507224291563034,
  0.18449550867080688,
  0.7271752953529358,
  0.15585480630397797,
  0.10400670766830444,
  0.27948102355003357,
  0.1779264211654663],
 'fear': [0.9281681180000305,
  0.9425276517868042,
  0.9723207950592041,
  0.3607056736946106,
  0.09504345804452896,
  0.05136283487081528,
  0.7474274635314941,
  0.404496967792511,
  0.9155239462852478,
  0.05136283487081528],
 'joy': [0.9327981472015381,
  0.7044221758842468,
  0.7672379016876221,
  0.25188156962394714,
  0.04056442901492119,
  0.04337584972381592,
  0.8725655674934387,
  0.04056442901492119,
  0.04056442901492119,
  0.04056442901492119],
 'sadness': [0.6462159156799316,
  0.8879393339157104,
  0.5494766

In [24]:
from tqdm import tqdm

emotion_labels = ["anger", "disgust", "fear", "joy", "sadness", "surprise", "neutral"]
isbns= []
emotion_scores = {label: [] for label in emotion_labels}


for i in tqdm(range(len(books))):
    isbns.append(books["isbn13"][i])
    sentences = books["description"][i].split(".")
    predictions = classifier(sentences)
    max_scores = calculate_max_emotion_scores(predictions)
    for label in emotion_labels:
        emotion_scores[label].append(max_scores[label])

100%|██████████| 5197/5197 [11:12<00:00,  7.73it/s]


In [25]:
emotions_df = pd.DataFrame(emotion_scores)
emotions_df["isbn13"] = isbns

In [26]:
books = pd.merge(books, emotions_df, on ="isbn13")

In [28]:
books.to_csv("../data/books_with_emotions.csv", index = False)